Coul look like:

1. For each order 1 peak, select the TES for which it can be measured
2. Measure by hand its position for each TES for each order 1 peak
3. Define a frequency resolution and create the theoretical synthbeam profile for each frequency
4. At each position related to the frequency measured, get the flux in a zone on the order 1 peak and around at the same elevation
5. At each elevation, compute the atmosphere temperature and deduce the transmission
6. Fit all frequencies together with weights on the synth beam that are the Moon emission
7. Stack the results for all TES and all order 1 peaks

Questions/remarks:

- the pixel size determines the frequency dependency (?)
- for each pixel we get the corresponding frequency and we compare the signal with the QUBIC instrument beam at that frequency times the Moon emission (what about neighbour frequencies?)
- for each order 1 peak, I create a list with all TES that have it
- I note the order 1 peak position manually for all these
- the fit is applied on each pixel seperately and each order 1 peak seperately and I take the weighted mean or median of all TES? weights to be determined with std of flat space between orders 0 and 1
- to get a better understanding of atmosphere, measure flux at two other positions with same elevation?
- sort out the thing in Giuseppe's paper with the emission/transmission of atm: is it that we can understand the transmission of atm from the emission measured?
- how to know what is atmosphere temperature (atmosphere emission) and simply TES bath temperature changing? --> need to find the TES bath temperature data to remove this trend?
- the order 0 and order 1 peaks don't seem to have the same amplitude ratio as the theoretical synthbeam

## Imports and notebook configuration

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# %matplotlib inline
%matplotlib ipympl
# %matplotlib widget

### General imports
import os
import sys
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
import healpy as hp
import pickle

plt.rc('figure',figsize=(10, 6))
plt.rc('font',size=12)

### Astropy configuration
from astropy.visualization import quantity_support
quantity_support()
import astropy.units as u



#### QUBIC IMPORT
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument




def iQS2iQP(indexQS):
    qpnumi, qpasici = qp.pix2tes.pix2tes(indexQS+1)
    return qpnumi+(qpasici-1)*128-1

def iQP2iQS(indexQP):
    QStesnum = qp.pix2tes.tes2pix(indexQP%128+1, indexQP//128+1)
    return QStesnum-1

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)

# plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)


### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
# import fitting as fit
# import time_domain_tools as tdt
# import useful_functions as uf
import qubic.scripts.MoonProject.pipeline_moon_plotting as pmp
import qubic.scripts.MoonProject.pipeline_moon_functions as pmf

In [ ]:
mydatadir = '/Users/huchet/Documents/code/data/ComissioningTD/'
mydatadir2 = "/Users/huchet/Documents/code/scripts/MoonProject/"
mydatadir3 = "/Users/huchet/Documents/code/data/"

### Observation date and corresponding file

In [ ]:
ObsDate = '2022-07-14'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*')
print(dirs)
datadir = dirs[0]

### Observing Site

## Get good maps and Moon positions on TES in Moon coordinates

In [ ]:
# allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned.pkl", "rb" ) )
allTESNum, allmaps, _, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_raw.pkl", "rb" ) )

In [ ]:
# allTESNum, allmaps_2, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_raw.pkl", "rb" ) )

In [ ]:
_, _, moon_fit, _ = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned.pkl", "rb" ) )

In [ ]:
# np.all(allmaps == allmaps_2)

### Observing Site

In [ ]:
Salta_CNEA = {'lat':-24.731358*u.deg,
              'lon':-65.409535*u.deg,
              'height':1152*u.m,
              'UTC_Offset':-3*u.hour}
Obs_Site = Salta_CNEA
data=None

In [ ]:
az_qubic = 116.4
start_tt = 10000
speedmin = 0.1

try:
    data
except:
    data, tt, alltod, azt, elt, newazt, newelt, scantype = pmf.format_data(az_qubic, start_tt, Obs_Site, speedmin, data, datadir)

In [ ]:
moon_azt = np.array([moon_fit[i][0][0] for i in range(len(moon_fit))])
moon_elt = np.array([moon_fit[i][0][1] for i in range(len(moon_fit))])
no_moon_azt = moon_azt - 3 # 3 degrees from the Moon, same elevation

In [ ]:
print(moon_azt, moon_elt)

In [ ]:
%%script echo skipping

# compare the signal in a circle on the Moon and a circle next to it

sample_radius = 0.25 # degree

S_moon = np.zeros(len(moon_fit))
S_atm = np.zeros_like(S_moon)
for i in range(len(moon_fit)):
    if i != 151: #not visibly_ok_arr[i]:
        continue
    print("TES {}".format(i + 1))
    _, _ , tod_i = pmf.make_coadded_maps_TES(tt, alltod[i], azt, elt, scantype, newazt, newelt, nside=256, doplot=False, check_back_forth=False, also_tod=True)
    mask_elt = (elt >= moon_elt[i] - sample_radius) & (elt <= moon_elt[i] + sample_radius)
    mask_moon = mask_elt & (azt >= moon_azt[i] - sample_radius) & (elt <= moon_azt[i] + sample_radius)
    mask_no_moon = mask_elt & (azt >= no_moon_azt[i] - sample_radius) & (elt <= no_moon_azt[i] + sample_radius)
    S_moon[i] = np.mean(tod_i[mask_moon])
    S_atm[i] = np.mean(tod_i[mask_no_moon])

In [ ]:
NumTES = 73 # 73, 152, 64 (ordre pics différent)
iTES = NumTES - 1

In [ ]:
hp.gnomview(allmaps[iTES], rot=[0,0], reso=8, title='TES {}'.format(NumTES))
hp.graticule()
hp.projscatter(0, 0, marker='x', c="white")
hp.projplot([np.linspace(-2, 2), np.linspace(-3, 3)], marker='x', c="white")
plt.tight_layout()
plt.show()
print(moon_fit[iTES])

In [ ]:
def get_great_circle_traj(point_A, point_B, sphere_centre, sphere_radius, npoints, delta_angle_deg):
    if (not np.isclose(np.linalg.norm(point_A - sphere_centre), sphere_radius)) or (not np.isclose(np.linalg.norm(point_B - sphere_centre), sphere_radius)):
        print(np.linalg.norm(point_A - sphere_centre))
        print(np.linalg.norm(point_B - sphere_centre))
        print(sphere_radius)
        raise ValueError("One of the points is not on the sphere.")
    delta_angle_rad = np.radians(delta_angle_deg)
    plane = get_plane(point_A, point_B, sphere_centre) # compute the equation of the plane with A, B and the sphere centre C
    vec_1, vec_2 = get_vect_plane(plane, point_A, sphere_centre) # create two orthogonal vectors, one pointing at A
    vec_A = point_A - sphere_centre
    vec_B = point_B - sphere_centre
    angle = np.arccos(np.dot(vec_A, vec_B)/(np.linalg.norm(vec_A) * np.linalg.norm(vec_B))) # compute the angle between CA and CB
    theta = np.linspace(0 - delta_angle_rad, angle + delta_angle_rad, npoints).reshape(npoints, 1)
    great_circle = sphere_radius * (np.cos(theta) * vec_1 + np.sin(theta) * vec_2) # get the geodesic on the sphere between the two points, +/- delta_angle_deg
    return great_circle

def get_plane(point_A, point_B, point_C): # plane equation ax + by + cz + d = 0
    vec_1 = (point_A - point_C)
    vec_2 = (point_B - point_C)
    perp_vect = np.cross(vec_1, vec_2)
    a, b, c = perp_vect
    d = -np.dot(perp_vect, point_A)
    return np.array([a, b, c, d])

def get_vect_plane(plane, point_ref, sphere_centre): # get two orthogonal vectors on the plane, one pointing at point_ref from the sphere_centre
    perp_vect = plane[:3]/np.sqrt(np.sum(plane[:3]**2))
    vec_1 = (point_ref - sphere_centre)/np.linalg.norm(point_ref - sphere_centre)
    vec_2 = np.cross(perp_vect, vec_1)/np.linalg.norm(np.cross(vec_1, perp_vect))
    return vec_1, vec_2

def spherical2cartesian(rho, theta, phi):
    x = rho * np.sin(theta) * np.cos(phi)
    y = rho * np.sin(theta) * np.sin(phi)
    z = rho * np.cos(theta)
    return x, y, z

In [ ]:
def get_traj(point_A_sph, point_B_sph, npoints, delta_angle_deg):
    point_A = spherical2cartesian(1, point_A_sph[0], point_A_sph[1])
    point_B = spherical2cartesian(1, point_B_sph[0], point_B_sph[1])
    sphere_centre=np.array([0, 0, 0])
    
    great_circle_traj = get_great_circle_traj(point_A, point_B, sphere_centre=sphere_centre, sphere_radius=1, npoints=npoints, delta_angle_deg = delta_angle_deg)
    theta_gc = np.arccos(great_circle_traj[:, 2])
    phi_gc = np.arctan2(great_circle_traj[:, 1], great_circle_traj[:, 0])
    return theta_gc, phi_gc

In [ ]:
def get_values_line(map_, point_A_sph, point_B_sph, npoints, delta_angle_deg):
    theta_gc, phi_gc = get_traj(point_A_sph, point_B_sph, npoints, delta_angle_deg)
    return hp.get_interp_val(map_, theta_gc, phi_gc), theta_gc, phi_gc


def get_values_large_line(map_, theta_gc, phi_gc, width_angle_deg, nlines):
    theta_gc_ = np.tile(theta_gc, (nlines, 1))
    phi_gc_ = phi_gc + np.radians(np.linspace(-width_angle_deg, width_angle_deg, nlines)).reshape(nlines, 1)
    res = hp.get_interp_val(map_, theta_gc_, phi_gc_)
    return np.mean(res, axis=0), res, theta_gc_, phi_gc_


In [ ]:
order_0_pos = moon_fit[iTES][0]

if NumTES == 73:
    # order_1_pos_start = np.array([2.78, -1.74])
    order_1_pos_end = np.array([4.77, +0.02])
elif NumTES == 152:
    order_1_pos_end = np.array([-7.60, +6.63])
elif NumTES == 99:
    order_1_pos_end = np.array([3.11, -3.25])
elif NumTES == 64:
    order_1_pos_end = np.array([4.57, -0.31])
print(order_1_pos_end - order_0_pos)

In [ ]:
all_az = np.array([pos[0] for pos in [order_0_pos, order_1_pos_end]])
all_el = np.array([pos[1] for pos in [order_0_pos, order_1_pos_end]])
order_1_pos_end

In [ ]:
# el = a * az + b
a = (np.mean(all_el[1:]) - all_el[0])/(np.mean(all_az[1:]) - all_az[0])
b = all_el[0] - a * all_az[0]

In [ ]:
delta_az = 1
x = np.linspace(moon_fit[iTES][0][0] + delta_az, order_1_pos_end[0] - delta_az, 200) # azimuth
y = a*x + b

# Extract the values along the line, using cubic interpolation
zi = hp.get_interp_val(allmaps[iTES], x, y, lonlat=True)

extra_shift = np.radians(0.05)

order_0_pos_sph = np.array([np.radians(90 - order_0_pos[1]), np.radians(order_0_pos[0])]) - extra_shift
order_1_pos_end_sph = np.array([np.radians(90 - order_1_pos_end[1]), np.radians(order_1_pos_end[0])]) - extra_shift

print(order_0_pos)
print(np.degrees(order_0_pos_sph))
# aze
zi_test, theta_gc_test, phi_gc_test = get_values_line(allmaps[iTES], order_0_pos_sph, order_1_pos_end_sph, npoints=200, delta_angle_deg=1)
# _, theta_gc_test, phi_gc_test = get_values_line(allmaps[iTES], order_0_pos_sph, order_1_pos_end_sph, npoints=200, delta_angle_deg=1)
# zi_test, zi_test_full, theta_gc_test_full, phi_gc_test_full = get_values_large_line(allmaps[iTES], theta_gc_test, phi_gc_test, width_angle_deg=0.1, nlines=3)

nlines = 4
theta_gc_test_diff = np.tile(theta_gc_test, (nlines, 1))
phi_gc_test_diff_up = phi_gc_test + np.radians(np.linspace(2, 3, nlines)).reshape(nlines, 1)
phi_gc_test_diff_down = phi_gc_test - np.radians(np.linspace(2, 3, nlines)).reshape(nlines, 1)
zi_test2 = hp.get_interp_val(allmaps[iTES], theta_gc_test_diff, phi_gc_test_diff_up)
zi_test3 = hp.get_interp_val(allmaps[iTES], theta_gc_test_diff, phi_gc_test_diff_down)

#-- Plot...
hp.gnomview(allmaps[iTES], reso=8, rot=(0, 0, 0), return_projected_map=True, no_plot=False).data
hp.projscatter(x, y, marker='.', c="r", lonlat=True)
hp.projscatter(theta_gc_test, phi_gc_test, marker='.', c="g", lonlat=False)
hp.projscatter(theta_gc_test, phi_gc_test + np.radians(2), marker='.', c="b", lonlat=False)
hp.projscatter(theta_gc_test, phi_gc_test - np.radians(2), marker='.', c="b", lonlat=False)
# hp.projscatter(theta_gc_test_full[0], phi_gc_test_full[0], marker='.', c="b", lonlat=False)
plt.show()

In [ ]:
zi_test2_mean = np.median(zi_test2, axis=0)
zi_test3_mean = np.median(zi_test3, axis=0)
concatenated_zi = np.concatenate([zi_test2, zi_test3])
concatenated_zi[concatenated_zi**2 > 1e20] = np.NaN
zi_test_diff = zi_test - np.mean(concatenated_zi, axis=0)
fig, ax = plt.subplots()
# ax.plot(zi)
# ax.plot(zi_test, label="zi_test")
# ax.plot(zi_test2_mean, label="zi_test2")
# ax.plot(zi_test3_mean, label="zi_test3")
ax.plot(zi_test_diff, label="zi_test_diff")
plt.legend()
plt.show()

In [ ]:
%%script echo skipping
plt.figure()
plt.plot(all_az, all_el)
plt.scatter(all_az, all_el)
plt.plot(all_az, a *all_az + b)
plt.show()

In [ ]:
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Qscene import QubicScene
import scipy.constants as cst

In [ ]:
dictfilename = 'qubic/qubic/dicts/global_source_oneDet.dict'
d = qubicDict()
d.read_from_file(dictfilename)
d['config'] = 'TD'
d['filter_nu']= 150000000000.0
d['beam_shape'] = 'gaussian'  # can be 'gaussian', 'fitted_beam' or 'multi_freq'  
d['synthbeam'] = None         # we put nothing
d['nside'] = 512              # To have nice SB maps
d['use_synthbeam_fits_file'] = False
d['synthbeam_fraction'] = 1
d['synthbeam_kmax'] = 3
q_instrument = QubicInstrument(d)
print(len(q_instrument.horn.center))


In [ ]:
# distance between two horns
delta_h = 14e-3 # m

In [ ]:
# index in arrays
order_0 = 24
order_1 = np.array([23, 30, 31, 32, 25, 18, 17, 16])

In [ ]:
def find_pos(size_pix, npix, pos, centre): # translate a position in physical units to a position in degrees
    return ((pos - (centre - npix * size_pix / 2 ))//size_pix).astype(int)

In [ ]:
def synth_beam(B_sky, B_det, pos_r, pos_n, lbda, P, f, delta_h, size_pix, npix_integration): # only one pos_n or only one pos_r
    x, y = np.radians(pos_r[..., 0]), np.radians(pos_r[..., 1])
    nx, ny = np.radians(pos_n[..., 0]), np.radians(pos_n[..., 1])
    npix = np.array(np.shape(B_sky))
    centre = np.array([0, 0])
    pix_n = find_pos(size_pix, npix, pos_n, centre)
    pix_r = find_pos(size_pix, npix, pos_r, centre)

    # print("pix_n", pix_n)
    # print(B_det[pix_n[..., 0], pix_n[..., 1]])

    res_before_int = B_sky[pix_r[..., 0], pix_r[..., 1]] * B_det[pix_n[..., 0], pix_n[..., 1]] * np.sin(P*np.pi*delta_h/lbda*(x/f - nx))**2 * np.sin(P*np.pi*delta_h/lbda*(y/f - ny))**2 / (np.sin(np.pi*delta_h/lbda*(x/f - nx))**2 * np.sin(np.pi*delta_h/lbda*(y/f - ny))**2)
    intermediate_sum = np.cumsum(res_before_int, axis=1) # sum over columns
    intermediate_sum = intermediate_sum[:, ::npix_integration] # get one out of npix_integration
    final_npix = npix//npix_integration
    intermediate_sum[:, 1 : final_npix[1]] = intermediate_sum[:, 1 : final_npix[1]] - intermediate_sum[:, 0 : final_npix[1] - 1] # this is the sum over npix_integration columns
    intermediate_sum = np.cumsum(intermediate_sum, axis=0)
    intermediate_sum = intermediate_sum[::npix_integration] # get one out of npix_integration
    intermediate_sum[1 : final_npix[0]] = intermediate_sum[1 : final_npix[0]] - intermediate_sum[0 : final_npix[0] - 1] # this is the sum over npix_integration columns
    integrated_res = intermediate_sum / npix_integration**2
    return integrated_res

In [ ]:
npix_x = 501
npix_y = 501

size_img = 30 # degrees
size_pix = size_img/npix_x

print("Size image is {} degrees".format(size_img))

In [ ]:
npix_integration = 10 # more pixels to then integrate
npix_x_int = npix_x * npix_integration
npix_y_int = npix_y * npix_integration
size_pix_int = size_pix / npix_integration

In [ ]:
reso = [12.9 / size_pix_int] # fwhm in pixels
xc, yc = npix_x_int//2, npix_y_int//2 # centre of the map in pixel
B_sky = pmf.gauss2D(npix_x_int, npix_y_int, xc, yc, reso, amp=None, normal=True)
B_det = pmf.gauss2D(npix_x_int, npix_y_int, xc, yc, reso, amp=None, normal=True)

In [ ]:
%%script echo skipping

plt.figure()
plt.imshow(B_sky)
plt.colorbar()
plt.show()

plt.figure()
plt.plot(B_sky[npix_x_int//2])
plt.show()


In [ ]:
print(size_pix)
print(size_pix_int)

In [ ]:
%%script echo skipping

mean_nu = 150e9
bw = 0.15*mean_nu
nus = np.linspace(mean_nu - bw, mean_nu + bw, 10)
lmbdas = cst.c/nus

# There might be a problem between x and y
# xx, yy = np.meshgrid(np.linspace(-npix_x//2, npix_x//2, npix_x_int)*size_pix, np.linspace(-npix_y//2, npix_y//2, npix_y_int)*size_pix)
# xx, yy = 30, 0.1
# print(q_instrument.horn.center[iTES])
det_pos = q_instrument.horn.center[iTES]
xx, yy = det_pos[0]/size_pix, det_pos[1]/size_pix
# nx, ny = 30, 0.1 # why does the synthbeam change so much with nx
nx, ny = np.meshgrid(np.linspace(-npix_x//2, npix_x//2, npix_x_int)*size_pix, np.linspace(-npix_y//2, npix_y//2, npix_y_int)*size_pix)

pos_n = np.array([nx, ny])
pos_r = np.array([xx, yy])
# pos_r = np.swapaxes(pos_r, 0, 2)
pos_n = np.swapaxes(pos_n, 0, 2)

sb = []
sb_tot = np.zeros((npix_x, npix_y))
for i_lmbda, lmbda in enumerate(lmbdas):
    sb.append(synth_beam(B_sky, B_det, pos_r, pos_n, lmbda, P=len(q_instrument.horn.center), f=q_instrument.optics.focal_length, delta_h=delta_h, size_pix=size_pix, npix_integration=npix_integration))
    sb_tot += sb[i_lmbda]

In [ ]:
%%script echo skipping

print(np.shape(sb))
print(np.min(sb), np.max(sb), np.mean(sb), np.std(sb))

In [ ]:
%%script echo skipping

plt.figure()
max_sb = np.max(sb_tot)
plt.imshow(sb_tot/max_sb, norm="log", vmin=1e-2, vmax=1e0)
# plt.imshow(sb_tot)#, vmin=-max_sb, vmax=max_sb)
plt.colorbar()
plt.show()

plt.figure()
plt.imshow(sb[0])
plt.show()

In [ ]:
central_freq = 150.
numin = central_freq * (1 - d['filter_relative_bandwidth']/2.)
numax = central_freq * (1 + d['filter_relative_bandwidth']/2.)
n_nus = 5
nus = np.linspace(numin, numax, n_nus)
print(nus)

thetas = np.zeros((n_nus, 248, (2*d['synthbeam_kmax']+1)**2))
phis = np.zeros((n_nus, 248, (2*d['synthbeam_kmax']+1)**2))
vals = np.zeros((n_nus, 248, (2*d['synthbeam_kmax']+1)**2))
my_sbs = np.zeros((n_nus, 12*d['nside']**2))
for i_nu in range(n_nus):
    print(nus[i_nu])
    d['filter_nu'] = nus[i_nu] * 1e9
    q_instrument_i = QubicInstrument(d)
    q_scene_i = QubicScene(d)
    print(d["synthbeam_kmax"])
    thetas[i_nu, :, :], phis[i_nu, :, :], vals[i_nu, :, :] = q_instrument_i._peak_angles_unsorted(q_scene_i, 
                                                d['filter_nu'], 
                                                q_instrument_i.detector.center, 
                                                q_instrument_i.synthbeam, 
                                                q_instrument_i.horn, 
                                                q_instrument_i.primary_beam)
    
    my_sbs[i_nu, :] = q_instrument_i.get_synthbeam(q_scene_i, iTES)

In [ ]:
inu = 0
# point_A = spherical2cartesian(1, lon[inu, iTES, 0], lat[inu, iTES, 0])
# point_B = spherical2cartesian(1, lon[inu, iTES, 1], lat[inu, iTES, 1])
ind_0 = order_0
ind_1 = order_1[0]
point_A = spherical2cartesian(1, thetas[inu, iTES, ind_0], phis[inu, iTES, ind_0])
point_B = spherical2cartesian(1, thetas[inu, iTES, ind_1], phis[inu, iTES, ind_1])
sphere_centre=np.array([0, 0, 0])

print(point_A)
print(point_B)

great_circle_traj = get_great_circle_traj(point_A, point_B, sphere_centre=sphere_centre, sphere_radius=1, npoints=200, delta_angle_deg=1)
theta_gc = np.arccos(great_circle_traj[:, 2])
phi_gc = np.arctan2(great_circle_traj[:, 1], great_circle_traj[:, 0])

print(np.degrees(theta_gc[0]))
print(np.degrees(phi_gc[2]))

In [ ]:
lon, lat = hp.pixelfunc.thetaphi2lonlat(thetas, phis)
# print(lon[inu, iTES])
# print(lat[inu, iTES])

two_peaks_az = lon[inu, iTES, [ind_0, ind_1]] + 180
two_peaks_el = lat[inu, iTES, [ind_0, ind_1]] + 180

# el = a * az + b
a_2 = (two_peaks_el[1] - two_peaks_el[0])/(two_peaks_az[1] - two_peaks_az[0])
b_2 = two_peaks_el[0] - a_2 * two_peaks_az[0]

delta_az = 2
x_2 = np.linspace(two_peaks_az[0] - delta_az, two_peaks_az[1] + delta_az, 200) # azimuth
y_2 = a_2*x_2 + b_2

x_2 -= 180
y_2 -= 180

hp.gnomview(np.log10(my_sbs[inu]/np.max(my_sbs[inu])), rot=[0,90], reso=20, min=-5, max=0,
             title='Theory {} {} GHz: TES #{}'.format(d['config'], d['filter_nu']/1e9,iTES))
variation = vals[inu, iTES, :]/np.max(vals[inu, iTES, :])
hp.projscatter(thetas[inu, iTES, :], phis[inu, iTES, :], c=variation, alpha=variation**(1/4), marker='x', cmap='Reds')
hp.projscatter(x_2, y_2, marker='o', c="r", lonlat=True)
hp.projscatter(theta_gc, phi_gc, marker='x', c="g")
# for i_peak in range(np.shape(thetas)[2]):
#     hp.projtext(thetas[inu, iTES, i_peak], phis[inu, iTES, i_peak], 
#                 '{0:5.0f}: {1:4.2f}'.format(i_peak, vals[inu, iTES,i_peak]/np.max(vals[inu, iTES,:])), c='w', fontsize=8)
plt.show()

In [ ]:
%%script echo skipping

# Extract the values along the line, using cubic interpolation
# zi_2 = hp.get_interp_val(my_sbs[inu], x_2, y_2, lonlat=True)
zi_2 = hp.get_interp_val(my_sbs[inu], theta_gc, phi_gc)
zi_0 = hp.get_interp_val(my_sbs[-1], theta_gc, phi_gc)
print(np.shape(zi_2))

# fig, ax = plt.subplots()
# ax.plot(x_2, y_2)
# plt.show()

fig, ax = plt.subplots()
ax.plot(zi_0)
ax.plot(zi_2)
plt.show()

In [ ]:
sb = np.sum(my_sbs, axis=0)
hp.gnomview(np.log10(sb/np.max(sb)), rot=[0,90], reso=20, min=-5, max=0,
             title='Theory {} {} GHz: TES #{}'.format(d['config'], d['filter_nu']/1e9, iTES + 1))
for i_nu in range(n_nus):
    variation = vals[i_nu, iTES,:]/np.max(vals[i_nu, iTES,:])
    hp.projscatter(thetas[i_nu, iTES,:], phis[i_nu, iTES,:], c=variation, alpha=variation**(1/4), marker='x', cmap='Reds')
    for i_peak in range(np.shape(thetas)[2]):
        hp.projtext(thetas[inu, iTES, i_peak], phis[inu, iTES, i_peak], 
                    '{0:5.0f}: {1:4.2f}'.format(i_peak, vals[i_nu, iTES,i_peak]/np.max(vals[i_nu, iTES,:])), c='w', fontsize=8)


In [ ]:
# get theoretical values along line in thetas phis

# zi_3, theta_gc_3, phi_gc_3 = get_values_line(my_sbs[inu], [thetas[inu, iTES, 0], phis[inu, iTES, 0]], [thetas[inu, idet, 1], phis[inu, iTES, 1]], npoints=200, delta_angle_deg=1)
# zi_3, theta_gc_3, phi_gc_3 = get_values_line(sb, [thetas[inu, iTES, 1], phis[inu, iTES, 1]], [thetas[inu, iTES, 0], phis[inu, iTES, 0]], npoints=200, delta_angle_deg=1)
sb_conv = hp.smoothing(sb, fwhm=np.radians(0.5))
ind_0 = order_0
ind_1 = order_1[0]
zi_3, theta_gc_3, phi_gc_3 = get_values_line(sb_conv, [thetas[inu, iTES, ind_0], phis[inu, iTES, ind_0]], [thetas[inu, iTES, ind_1], phis[inu, iTES, ind_1]], npoints=200, delta_angle_deg=1)
print(np.shape(zi_3))

# fig, ax = plt.subplots()
# ax.plot(x_2, y_2)
# plt.show()

fig, ax = plt.subplots()
ax.plot(zi_3)
plt.show()

In [ ]:
theta_gc2 = theta_gc + np.radians(4)
hp.gnomview(np.log10(sb/np.max(sb)), rot=[0,90], reso=20, min=-5, max=0,
             title='Theory {} {} GHz: TES #{}'.format(d['config'], d['filter_nu']/1e9,iTES))
variation = vals[inu, iTES,:]/np.max(vals[inu, iTES,:])
hp.projscatter(thetas[inu, iTES,:], phis[inu, iTES,:], c=variation, alpha=variation**(1/4), marker='x', cmap='Reds')
hp.projscatter(x_2, y_2, marker='o', c="r", lonlat=True)
hp.projscatter(theta_gc, phi_gc, marker='x', c="g")
hp.projscatter(theta_gc2, phi_gc, marker='x', c="b")
for i_peak in range(np.shape(thetas)[2]):
        hp.projtext(thetas[inu, iTES, i_peak], phis[inu, iTES, i_peak], 
                    '{0:5.0f}: {1:4.2f}'.format(i_peak, vals[i_nu, iTES,i_peak]/np.max(vals[i_nu, iTES,:])), c='w', fontsize=8)
plt.show()

In [ ]:
print(np.shape(great_circle_traj))

In [ ]:
%%script echo skipping

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot3D(great_circle_traj[:, 0], great_circle_traj[:, 1], great_circle_traj[:, 2], 'blue')
ax.scatter3D(point_A[0], point_A[1], point_A[2], "red")
ax.scatter3D(point_B[0], point_B[1], point_B[2], "red")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
# ax.plot(zi_2/np.max(zi_2))
# ax.plot(zi/np.max(zi))
ax.plot(zi_3/np.max(zi_3), label="smoothed synthbeam at Moon fwhm")
ax.plot(zi_test/np.max(zi_test), label="order 0 and 1 Moon peaks")
ax.plot(zi_test_diff/np.nanmax(zi_test_diff), label="same - neighbours")
plt.legend()
plt.show() 

In [ ]:
print(q_instrument.filter.bandwidth*1e-9)
print(q_instrument.filter.relative_bandwidth)

In [ ]:
peak_8_list = [False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
 False,  True,  True, False, False, False, False, False,  True, False, False, False,
 False, False, False,  True, False, False, False, False, False,  True, False, False,
 False, False,  True,  True, False, False, False, False,  True, False, False, False,
  True,  True, False,  True, False, False,  True, False, False, False, False, False,
  True, False, False, False, False, False, False, False,  True, False,  True, False,
 False,  True,  True, False, False, False, False, False, False, False, False, False,
 False, False,  True, False, False, False, False, False, False, False,  True, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False,]

In [ ]:
doplot = False

npoints = 200
delta_angle_deg = 1 # take a bit more at each end of line
extra_shift = np.radians(0.05) # radians # shift applied on real data positions to fit the theoretical positions (why?)

central_freq = 150.
numin = central_freq * (1 - d['filter_relative_bandwidth']/2.)
numax = central_freq * (1 + d['filter_relative_bandwidth']/2.)
n_nus = 5
nus = np.linspace(numin, numax, n_nus)
nside = 256
npix = 12 * d["nside"]**2
ndet = np.sum(peak_8_list)
all_values_sbs = np.zeros((npoints, ndet))
mean_val_sb = np.zeros((npoints))

ind_0 = order_0 # index of order 0 in thetas, phis, vals arrays
ind_1 = order_1[0] # index of one of the order 1 in thetas, phis, vals arrays

values_mean = np.zeros(npoints)
idet = -1
for i_TES_, isokTES in enumerate(peak_8_list):
    if not isokTES:
        continue
    idet += 1
    map_i = allmaps[i_TES_]
    order_0_pos = moon_fit[i_TES_][0]
    order_1_pos_end = order_0_pos + (np.array([7.35, -6.17])) # test of a general direction for all TES
    order_0_pos_sph = np.array([np.radians(90 - order_0_pos[1]), np.radians(order_0_pos[0])]) - extra_shift
    order_1_pos_end_sph = np.array([np.radians(90 - order_1_pos_end[1]), np.radians(order_1_pos_end[0])]) - extra_shift
    # zi_test, theta_gc_test, phi_gc_test = get_values_line(map_i, order_0_pos_sph, order_1_pos_end_sph, npoints=200, delta_angle_deg=delta_angle_deg)
    _, theta_gc_test, phi_gc_test = get_values_line(map_i, order_0_pos_sph, order_1_pos_end_sph, npoints=200, delta_angle_deg=delta_angle_deg)
    zi_test, _, _, _ = get_values_large_line(map_i, theta_gc_test, phi_gc_test, width_angle_deg=0.1, nlines=3)
    nlines = 4
    theta_gc_test_diff = np.tile(theta_gc_test, (nlines, 1))
    phi_gc_test_diff_up = phi_gc_test + np.radians(np.linspace(2, 3, nlines)).reshape(nlines, 1)
    phi_gc_test_diff_down = phi_gc_test - np.radians(np.linspace(2, 3, nlines)).reshape(nlines, 1)
    zi_test2 = hp.get_interp_val(map_i, theta_gc_test_diff, phi_gc_test_diff_up)
    zi_test3 = hp.get_interp_val(map_i, theta_gc_test_diff, phi_gc_test_diff_down)
    zi_test2_mean = np.median(zi_test2, axis=0)
    zi_test3_mean = np.median(zi_test3, axis=0)
    concatenated_zi = np.concatenate([zi_test2, zi_test3])
    concatenated_zi[concatenated_zi**2 > 1e20] = np.NaN
    mean_concatenated_zi = np.mean(concatenated_zi, axis=0)
    zi_test_diff = zi_test - mean_concatenated_zi
    values_mean += zi_test

    if doplot:
        hp.gnomview(map_i, reso=8, rot=(0, 0, 0), return_projected_map=True, no_plot=False).data
        hp.projscatter(theta_gc_test, phi_gc_test, marker='.', c="g", lonlat=False)
        hp.projscatter(theta_gc_test, phi_gc_test + np.radians(2), marker='.', c="b", lonlat=False)
        hp.projscatter(theta_gc_test, phi_gc_test - np.radians(2), marker='.', c="b", lonlat=False)
        plt.show()

    sb_idet = np.zeros((npix))
    for inu in range(n_nus):
        d['filter_nu'] = nus[inu] * 1e9
        q_instrument_i = QubicInstrument(d)
        q_scene_i = QubicScene(d)
        sb_idet += q_instrument_i.get_synthbeam(q_scene_i, i_TES_)
    sb_smooth_i = hp.smoothing(sb_idet, fwhm=np.radians(0.5))
    # values_sb_i, _, _ = get_values_line(sb_smooth_i, [thetas[0, i_TES_, ind_0], phis[0, i_TES_, ind_0]], [thetas[0, i_TES_, ind_1], phis[0, i_TES_, ind_1]], npoints=npoints, delta_angle_deg=delta_angle_deg)
    _, theta_gc_i, phi_gc_i = get_values_line(sb_smooth_i, [thetas[0, i_TES_, ind_0], phis[0, i_TES_, ind_0]], [thetas[0, i_TES_, ind_1], phis[0, i_TES_, ind_1]], npoints=npoints, delta_angle_deg=delta_angle_deg)
    values_sb_i, _, _, _ = get_values_large_line(sb_smooth_i, theta_gc_i, phi_gc_i, width_angle_deg=0.1, nlines=3)
    all_values_sbs[:, idet] = values_sb_i
    mean_val_sb += values_sb_i

    fig, ax = plt.subplots(figsize=(12, 6))
    plot_sb = all_values_sbs[:, idet]/n_nus
    ax.set_title("TES {}".format(i_TES_ + 1))
    ax.plot(plot_sb/np.max(plot_sb), label="smoothed synthbeam at Moon fwhm")
    ax.plot(zi_test/np.max(zi_test), label="order 0 and 1 Moon peaks")
    ax.plot(zi_test_diff/np.nanmax(zi_test_diff), label="same - neighbours")
    ax.plot(mean_concatenated_zi/np.max(zi_test), label="neighbours")
    plt.legend()
    plt.show() 

values_mean /= ndet
mean_val_sb /= ndet
sum_mean_val_sb = np.sum(mean_val_sb, axis=0)
